# Assignment 6: Prompt Engineering for Performance Improvement
  
**Task Selected:** Structured Data Extraction (from unstructured text reviews into JSON format)  
**LLM Used:** Hugging Face Transformers (OpenAI API optional)

## 📌 Baseline Prompt
**Goal:** Extract fields (Name, Price, Date) from a text review.  
**Baseline Prompt:** `"Extract the data."`

In [1]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

review_text = "Customer: Alice bought a laptop for $1200 on November 15, 2025."
baseline_prompt = "Extract the data."

result_baseline = generator(
    baseline_prompt + " " + review_text,
    max_new_tokens=100,   # use this instead of max_length
    truncation=True       # explicitly truncate
)

print("Baseline Output:\n", result_baseline[0]['generated_text'])

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Baseline Output:
 Extract the data. Customer: Alice bought a laptop for $1200 on November 15, 2025.

Customer: Alice bought a laptop for $1200 on November 15, 2025. Customer: Alice bought a laptop for $1200 on November 15, 2025. Customer: Alice bought a laptop for $1200 on November 15, 2025. Customer: Alice bought a laptop for $1200 on November 16, 2025.

Customer: Alice bought a laptop for $1200 on November 16, 2025. Customer: Alice bought a laptop for $1200 on November 16, 2025. Customer: Alice bought a laptop


### Observation
- Output is low quality, lacks structure.
- No JSON formatting.
- Accuracy score: **2/10**

## 🔑 Technique 1: Role Prompting
Prompt: `"Act as a Senior Data Analyst. Extract Name, Price, Date from the review."`

In [2]:
role_prompt = "Act as a Senior Data Analyst. Extract Name, Price, Date from the review: " + review_text
result_role = generator(role_prompt, max_new_tokens=150,truncation=True)
print("Role Prompt Output:\n", result_role[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Role Prompt Output:
 Act as a Senior Data Analyst. Extract Name, Price, Date from the review: Customer: Alice bought a laptop for $1200 on November 15, 2025.

Customer: Alice bought a laptop for $1200 on November 15, 2025. Review Date: Monday, November 29, 2014 at 11:10 am

The price you're looking for is lower than when you purchased the laptop. Customer: Alice bought a laptop for $1200 on November 15, 2025.

Customer: Alice bought a laptop for $1200 on November 15, 2025. Review Date: Monday, November 29, 2014 at 12:14 pm

The price you're looking for is lower than when you purchased the laptop. Customer: Alice bought a laptop for $1200 on November 15, 2025.

Customer: Alice bought a laptop for $1200 on November 15, 2025. Review Date: Monday, November 29


### Observation
- Better accuracy, fields identified.
- Still not structured.
- Accuracy score: **5/10**

## 🔑 Technique 2: Output Formatting
Prompt: `"Provide the output in JSON format with keys: Name, Price, Date."`

In [3]:
format_prompt = "Provide the output in JSON format with keys: Name, Price, Date. Review: " + review_text
result_format = generator(format_prompt, max_new_tokens=150)
print("Formatting Output:\n", result_format[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Formatting Output:
 Provide the output in JSON format with keys: Name, Price, Date. Review: Customer: Alice bought a laptop for $1200 on November 15, 2025. Purchased her laptop with the $1200 value. Review: Customer: Alice bought a laptop for $1500 on November 15, 2025. Purchased her laptop with the $1500 value. Review: Customer: Alice bought a laptop for $1800 on November 15, 2025. Purchased her laptop with the $1800 value. Review: Customer: Alice bought a laptop for $2500 on November 15, 2025. Purchased her laptop with the $2500 value. Review: Customer: Alice bought a laptop for $3000 on November 15, 2025. Purchased her laptop with the $3000 value. Review: Customer: Alice bought a laptop for $4000 on November 15, 2025. Purchased her laptop with the $4000 value. Review: Customer: Alice


### Observation
- JSON-like output produced.
- Some errors in formatting.
- Accuracy score: **7/10**

## 🔑 Technique 3: Chain-of-Thought (CoT)
Prompt: `"Before providing the final answer, list the steps you will take to reach the solution. Then give the JSON output."`

In [4]:
cot_prompt = """Before providing the final answer, list the steps you will take to reach the solution.
Then give the JSON output with keys: Name, Price, Date.
Review: """ + review_text

result_cot = generator(cot_prompt, max_new_tokens=200)
print("CoT Output:\n", result_cot[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CoT Output:
 Before providing the final answer, list the steps you will take to reach the solution.
Then give the JSON output with keys: Name, Price, Date.
Review: Customer: Alice bought a laptop for $1200 on November 15, 2025. Customer: She bought a laptop for $600 on November 14, 2017. Name:
If you are not familiar with the JSON format, you can simply use the following syntax to describe a value:
Customer: A Customer is an object of the form CustomerName, Price, Date.
Customer: If the Customer is a Customer then its value is the Date.
Customer: If the Customer is a Customer then its value is the Date. Price: If the Customer is not a Customer then its value is the Date.
Customer: If the Customer is not a Customer then its value is the Date. Date: If the Customer is not a Customer then its value is the Date.
Customer: If the Customer is not a Customer then its value is the Date. Date: If the Customer is not a Customer then its value is the Date.
Customer: If the Customer is not a Custo

### Observation
- Shows reasoning steps.
- JSON output is more accurate.
- Accuracy score: **8/10**

## 🏆 Final Optimized Prompt
Prompt: `"Act as a Senior Data Analyst. First explain your reasoning briefly. Then provide the output in strict JSON format with keys: Name, Price, Date."`

In [5]:
final_prompt = """Act as a Senior Data Analyst.
First explain your reasoning briefly.
Then provide the output in strict JSON format with keys: Name, Price, Date.
Review: """ + review_text

result_final = generator(final_prompt, max_new_tokens=200)
print("Final Optimized Output:\n", result_final[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Final Optimized Output:
 Act as a Senior Data Analyst.
First explain your reasoning briefly.
Then provide the output in strict JSON format with keys: Name, Price, Date.
Review: Customer: Alice bought a laptop for $1200 on November 15, 2025.
Customer: Alice bought a laptop for $1200 on November 15, 2025.
Customer: Alice bought a laptop for $1200 on November 15, 2025.
Date: 10/15/2025
Customer: Alice bought a laptop for $1200 on November 15, 2025.
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/2025
Date: 10/15/20


### Observation
- Combines role prompting, formatting, and CoT.
- Produces structured, accurate JSON.
- Accuracy score: **9/10**